### Unsupervised GRU-VAE training  

In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np # For standard deviation calculation
from modbus import ModbusDataset,ModbusFlowStream
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,recall_score
import torch.optim as optim
import pickle
import os 
from torch.utils.data import DataLoader
import time
import random
from utils import load_scalers
from random import SystemRandom
from sklearn.model_selection import train_test_split
import itertools
import torch.nn.init as init


def compute_threshold(mse_values):
    """
    Computes the anomaly detection threshold (for marking sample as Intrusion if the IS was greater )
    based on the mean and standard deviation of Mean Squared Error (MSE) values.
    Formula: thr = mean(MSE) + std(MSE)

    Args:
        mse_values (torch.Tensor or list/np.array): A tensor or list of MSE values
                                                    obtained from the validation set.

    Returns:
        float: The calculated threshold.
    """
    if not isinstance(mse_values, torch.Tensor):
        mse_values = torch.tensor(mse_values, dtype=torch.float32)

    if mse_values.numel() == 0:
        return 0.0 
    mean_mse = torch.mean(mse_values)
    std_mse = torch.std(mse_values)

    threshold = mean_mse + std_mse
    return threshold.item() 

def vae_loss_function(recon_x, x, mu, logvar,beta =1):
    """
    VAE loss function.
    """
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return (BCE + beta*KLD)


In [4]:
dataset_directory = "./dataset" # change this to the folder contain benign and attack subdirs
modbus = ModbusDataset(dataset_directory,"ready")
modbus.summary_print()

 The CIC Modbus Dataset contains network (pcap) captures and attack logs from a simulated substation network.
                The dataset is categorized into two groups: an attack dataset and a benign dataset
                The attack dataset includes network traffic captures that simulate various types of Modbus protocol attacks in a substation environment.
                The attacks are reconnaissance, query flooding, loading payloads, delay response, modify length parameters, false data injection, stacking Modbus frames, brute force write and baseline replay.
                These attacks are based of some techniques in the MITRE ICS ATT&CK framework.
                On the other hand, the benign dataset consists of normal network traffic captures representing legitimate Modbus communication within the substation network.
                The purpose of this dataset is to facilitate research, analysis, and development of intrusion detection systems, anomaly detection algorithms and

In [5]:
def _init_weights( module):
    if isinstance(module, nn.Linear):
        #Xavier Initialization
        init.xavier_normal_(module.weight)
        if module.bias is not None:
            init.zeros_(module.bias)
    return module
# AutoEncoder (AE)
class AE(nn.Module):
    """
    Encoder: (89-64-32)
    Decoder: (32-64-89)
    """
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(89, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 89),
            nn.ReLU()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon



In [111]:
a = AE()
first_iterator = list(a.parameters())
print(first_iterator[0])
print(first_iterator[0])

a.apply(_init_weights)
first_iterator = list(a.parameters())
print(first_iterator[0])
a.apply(_init_weights)
print(first_iterator[0])



Parameter containing:
tensor([[ 0.1040,  0.0537, -0.0960,  ..., -0.1059,  0.0405,  0.0280],
        [-0.0164,  0.0952, -0.0784,  ...,  0.0404,  0.0534,  0.0173],
        [-0.0385,  0.0915,  0.0383,  ...,  0.0031, -0.0315, -0.0096],
        ...,
        [ 0.1002, -0.0972, -0.0732,  ..., -0.0014,  0.0039, -0.0452],
        [-0.0682,  0.0007, -0.0890,  ...,  0.0590, -0.0071, -0.0748],
        [ 0.0239,  0.1055,  0.0341,  ..., -0.0421,  0.0008,  0.1047]],
       requires_grad=True)
Parameter containing:
tensor([[ 0.1040,  0.0537, -0.0960,  ..., -0.1059,  0.0405,  0.0280],
        [-0.0164,  0.0952, -0.0784,  ...,  0.0404,  0.0534,  0.0173],
        [-0.0385,  0.0915,  0.0383,  ...,  0.0031, -0.0315, -0.0096],
        ...,
        [ 0.1002, -0.0972, -0.0732,  ..., -0.0014,  0.0039, -0.0452],
        [-0.0682,  0.0007, -0.0890,  ...,  0.0590, -0.0071, -0.0748],
        [ 0.0239,  0.1055,  0.0341,  ..., -0.0421,  0.0008,  0.1047]],
       requires_grad=True)
Parameter containing:
tensor([[-0.

In [6]:

csv_files=[col for col in modbus.dataset["benign_dataset_dir"] if col.find("network-wide")!=-1]
sys_rand = SystemRandom()
sys_rand.shuffle(csv_files)
train_files,val_files = train_test_split(csv_files,test_size=0.2,shuffle=True)
test_files=[col for col in modbus.dataset["attack_dataset_dir"]["compromised-scada"] if col.find("ied1b")!=-1][0:4]
print("ied1b comp ied attack ->\n test: ",len(test_files),test_files)
print("network-wide number of csv files ->\n train :",len(train_files),train_files,"\n valid:",len(val_files),val_files)



ied1b comp ied attack ->
 test:  4 ['./dataset/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-6-labeled.csv', './dataset/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-3-labeled.csv', './dataset/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-4-labeled.csv', './dataset/ModbusDataset/attack/compromised-scada/ied1b/ied1b-network-captures/ready/vethc76bd3f-1-labeled.csv']
network-wide number of csv files ->
 train : 15 ['./dataset/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-23-labeled.csv', './dataset/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-17-labeled.csv', './dataset/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-14-labeled.csv', './dataset/ModbusDataset/benign/network-wide-pcap-capture/network-wide/ready/network-wide-normal-28-labeled.csv', './data

In [7]:
loaded_scalers = load_scalers("fitted_scalers")
AE_train_dataset=ModbusFlowStream( 
    shuffle=True,chunk_size=1,batch_size=64,csv_files=train_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=1
)
AE_train_dataloader=DataLoader(AE_train_dataset,batch_size=1,shuffle=False)
AE_val_dataloader=DataLoader(ModbusFlowStream( 
    shuffle=False,chunk_size=1,batch_size=64,csv_files=val_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=1
),batch_size=1,shuffle=False)
AE_test_dataloader=DataLoader(ModbusFlowStream(shuffle=False,chunk_size=1,batch_size=64,csv_files=test_files,scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=1),batch_size=1,shuffle=False)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
AE_model = AE().to(device)
lr = 0.01
wd= 1e-4
shuffle_files =True
AE_optimizer = optim.Adam(AE_model.parameters(), lr=lr, weight_decay=wd)
criterion = nn.MSELoss(reduction='sum').to(device)
eval_criterion = nn.MSELoss(reduction='none').to(device)



Successfully loaded scalers for 'network-wide'


In [8]:


def train_eval(model,train_dataloader,val_dataloader,test_dataloader,learning_rates= [0.01, 0.001, 1e-4, 1e-5],
               weight_decays=[1e-4, 1e-5, 1e-6],shuffle_files=True,epochs=2):
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    criterion = nn.MSELoss(reduction='sum').to(device)
    eval_criterion = nn.MSELoss(reduction='none').to(device)
    for lr, wd in itertools.product(learning_rates, weight_decays):
        print(f"\n================== Evaluate lr={lr}, wd={wd} ==================")
        model.apply(_init_weights)
        AE_optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        for epoch in range(epochs):
            time_1 = time.time()
            train_loss = 0
            model.train()
            if shuffle_files:
                sys_rand = SystemRandom()
                sys_rand.shuffle(AE_train_dataset.file_order_indices)
            for sequences, _ in train_dataloader:
                sequences=sequences.squeeze().to(device)
                AE_optimizer.zero_grad()

                if model._get_name()=="AE":
                    recon = model(sequences)
                    loss = criterion(recon, sequences) / sequences.size(0)
                elif model._get_name()=="VAE":
                    recon, mu, logvar = model(sequences)
                    loss = vae_loss_function(recon, sequences, mu, logvar)
                loss.backward()
                AE_optimizer.step()
                train_loss += loss.item()
            print(f"Train : time {(time.time()-time_1):.4f}",f"Epoch {epoch}", f"Train Loss: {train_loss / len(train_dataloader):.4f}")
            # Evaluate part
            if (epoch + 1) % 1 == 0:
                model.eval() 
                all_val_losses = []
                all_val_labels = []
                print(f"\n--- Running Evaluation for Epoch {epoch+1} lr ={lr} wd {wd} ---")
                with torch.no_grad():
                    for sequences, labels in val_dataloader:
                        sequences = sequences.squeeze().to(device)
                                
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE":
                            recon, mu, logvar = model(sequences)
                        val_loss = eval_criterion(recon, sequences)
                        if val_loss.dim() > 1:
                            intrusion_scores = val_loss.mean(dim=1)
                        else:
                            intrusion_scores = val_loss.unsqueeze(dim=0).mean(dim=1)
                        intrusion_scores = val_loss.mean(dim=1)
                        all_val_losses.extend(intrusion_scores.cpu().numpy())
                        all_val_labels.extend(labels.flatten().cpu().numpy())            
                threshold = compute_threshold(all_val_losses)
                print(f"Computed Threshold: {threshold:.4f}")
                all_val_losses = np.array(all_val_losses).squeeze()  
                all_val_labels = np.array(all_val_labels).squeeze()  
                # If intrusion score > threshold, predict 1 (intrusion), else 0 (benign)
                # For FDR, get True Positives (TP) and False Positives (FP)
                predictions = (all_val_losses > threshold).astype(int)
                accuracy = accuracy_score(all_val_labels, predictions)
                print(f"Val: Accuracy: {accuracy:.4f}  ")
                threshold=0.0023
                model.eval() 
                all_test_losses = []
                all_test_labels = []
                with torch.no_grad():
                    for sequences, labels in test_dataloader:
                        sequences = sequences.squeeze().to(device)
                        if model._get_name()=="AE":
                            recon = model(sequences)
                        elif model._get_name()=="VAE":
                            recon, mu, logvar = model(sequences)
                        test_loss = eval_criterion(recon, sequences)
                        if test_loss.dim() > 1:
                            intrusion_scores = test_loss.mean(dim=1)
                        else:
                            intrusion_scores = test_loss.unsqueeze(dim=0).mean(dim=1)
                        all_test_losses.extend(intrusion_scores.cpu().numpy())
                        all_test_labels.extend(labels.flatten().cpu().numpy())            
                all_test_losses = np.array(all_test_losses)
                all_test_labels = np.array(all_test_labels) 
                predictions = (all_test_losses > threshold).astype(int)
                binary_test_labels = (all_test_labels != 0).astype(int)
                accuracy = accuracy_score(binary_test_labels, predictions)
                f1 = f1_score(binary_test_labels, predictions, zero_division=0)
                recall = recall_score(binary_test_labels, predictions,zero_division=0)
                tn, fp, fn, tp = confusion_matrix(binary_test_labels, predictions, labels=[0, 1]).ravel()
                # FDR = FP / (FP + TP) 
                if (fp + tp) == 0:
                    fdr = 0.0 
                else:
                    fdr = fp / (fp + tp)
                print(f"Test : Accuracy: {accuracy:.4f} Recall : {recall:.4f} FDR: {fdr:.4f}  F1-score: {f1:.4f}  ")



In [104]:
train_eval(AE_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)



================== Evaluate lr=0.01, wd=0.0001 ==================
Train : time 109.1824 Epoch 0 Train Loss: 0.1841

--- Running Evaluation for Epoch 1 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0011
Val: Accuracy: 0.9950  
Test : Accuracy: 0.8894 Recall : 0.8442 FDR: 0.2954  F1-score: 0.7681  
Train : time 106.1758 Epoch 1 Train Loss: 0.0068

--- Running Evaluation for Epoch 2 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0009
Val: Accuracy: 0.9949  
Test : Accuracy: 0.8894 Recall : 0.8442 FDR: 0.2954  F1-score: 0.7681  

================== Evaluate lr=0.01, wd=1e-05 ==================
Train : time 100.7914 Epoch 0 Train Loss: 0.1683

--- Running Evaluation for Epoch 1 lr =0.01 wd 1e-05 ---
Computed Threshold: 0.0016
Val: Accuracy: 0.9954  
Test : Accuracy: 0.8894 Recall : 0.8443 FDR: 0.2954  F1-score: 0.7681  
Train : time 99.9983 Epoch 1 Train Loss: 0.0079

--- Running Evaluation for Epoch 2 lr =0.01 wd 1e-05 ---
Computed Threshold: 0.0012
Val: Accuracy: 0.9955  
Test : Accuracy: 0

In [9]:

# Variational AutoEncoder (VAE)
class VAE(nn.Module):
    """
    Encoder: (89-64-64-32 for mu and log_var)
    Decoder: (32-64-64-89)
    return x_recon, mu, logvar
    """
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(89, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(64, 32)
        self.fc_logvar = nn.Linear(64, 32)
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 89),
            nn.ReLU()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar



In [ ]:

VAE_model = VAE().to(device=device)
train_eval(VAE_model,AE_train_dataloader,AE_val_dataloader,AE_test_dataloader)


# for epoch in range(3):
#     time_1 = time.time()
#     train_loss = 0
#     AE_model.train()
#     if shuffle_files:
#         sys_rand = SystemRandom()
#         sys_rand.shuffle(AE_dataset.file_order_indices)
#     for sequences, _ in AE_dataloader:
#         sequences = sequences.squeeze().to(device)
#         VAE_optimizer.zero_grad()
#         recon, mu, logvar = VAE_model(sequences)
#         loss = vae_loss_function(recon, sequences, mu, logvar)
#         loss.backward()
#         VAE_optimizer.step()
#         train_loss += loss.item()
#     print("time",time.time()-time_1,f"Epoch {epoch}, Train Loss: {train_loss / len(AE_dataloader)}")



================== Evaluate lr=0.01, wd=0.0001 ==================
Train : time 141.5818 Epoch 0 Train Loss: 97.8765

--- Running Evaluation for Epoch 1 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0206
Val: Accuracy: 0.9378  
Test : Accuracy: 0.8535 Recall : 0.8524 FDR: 0.3822  F1-score: 0.7164  
Train : time 139.1011 Epoch 1 Train Loss: 93.5440

--- Running Evaluation for Epoch 2 lr =0.01 wd 0.0001 ---
Computed Threshold: 0.0382
Val: Accuracy: 0.8970  
Test : Accuracy: 0.8366 Recall : 0.8565 FDR: 0.4156  F1-score: 0.6948  

================== Evaluate lr=0.01, wd=1e-05 ==================
Train : time 135.4118 Epoch 0 Train Loss: 102.8192

--- Running Evaluation for Epoch 1 lr =0.01 wd 1e-05 ---
Computed Threshold: 0.0314
Val: Accuracy: 0.8984  
Test : Accuracy: 0.2174 Recall : 0.9999 FDR: 0.7829  F1-score: 0.3568  
Train : time 134.7249 Epoch 1 Train Loss: 102.6428

--- Running Evaluation for Epoch 2 lr =0.01 wd 1e-05 ---
Computed Threshold: 0.0322
Val: Accuracy: 0.8927  
Test : Accu

In [6]:

class AAE_Encoder(nn.Module):
    def __init__(self):
        """
        Encoder(Generator):(89-16-4-2)
        """
        super(AAE_Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(89, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 4),
            nn.LeakyReLU(0.2),
            nn.Linear(4, 2))
    def forward(self, x):
        return self.encoder(x)
class AAE_Decoder(nn.Module):
    def __init__(self):
        super(AAE_Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Linear(2, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 89),
            nn.LeakyReLU()
        )
    def forward(self, x):
        return self.decoder(x)
class AAE_Discriminator(nn.Module):
    def __init__(self):
        super(AAE_Discriminator, self).__init__()
        self.discriminator = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 2), # Output for binary classification (real/fake)
            nn.Sigmoid()
        )    
    def forward(self, x):
        return self.discriminator(x)


In [9]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aae_encoder = AAE_Encoder()
aae_decoder = AAE_Decoder()
aae_discriminator = AAE_Discriminator()
aae_encoder.to(device)
aae_decoder.to(device)
aae_discriminator.to(device)
optimizer_G = optim.Adam(list(aae_encoder.parameters()) + list(aae_decoder.parameters()), lr=1e-4)
optimizer_D = optim.Adam(aae_discriminator.parameters(), lr=1e-4)
adversarial_loss = nn.BCELoss(reduction="sum")
reconstruction_loss = nn.MSELoss(reduction="sum")


In [3]:
num_epochs=5
for epoch in range(num_epochs):
    aae_encoder.train()
    aae_decoder.train()
    aae_discriminator.train()
    if shuffle_files:
        sys_rand = SystemRandom()
        sys_rand.shuffle(AE_dataset.file_order_indices)
    for sequences,_ in AE_dataloader:
        sequences=sequences.squeeze().to(device)
        # 1) reconstruction + generator loss
        optimizer_G.zero_grad()
        fake_z = aae_encoder(sequences)
        decoded_seq = aae_decoder(fake_z)
        G_loss = 0.001*adversarial_loss(aae_discriminator(fake_z),  torch.ones(sequences.size(0), 2,device=device)) + 0.999*reconstruction_loss(decoded_seq, sequences)
        G_loss.backward()
        optimizer_G.step()
        # 2) discriminator loss
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(aae_discriminator(torch.randn(sequences.size(0), 2, device=device)),  torch.ones(sequences.size(0), 2,device=device))
        fake_loss = adversarial_loss(aae_discriminator(fake_z.detach()),  torch.zeros(sequences.size(0), 2,device=device))
        D_loss = 0.5*(real_loss + fake_loss)
        D_loss.backward()
        optimizer_D.step()
    # print loss
    print(
            "[Epoch %d/%d] [G loss: %f] [D loss: %f]"
            % (epoch, num_epochs, G_loss.item(), D_loss.item())
         )

NameError: name 'aae_encoder' is not defined

In [4]:
# GRU-VAE
class GRUVAE(nn.Module):
    """
    Gated Recurrent Unit : num_layers=2, hidden_size=256, dropout=0.01,window size (seq_len)= 40
    """
    def __init__(self, input_dim=89, hidden_dim=256, latent_dim=32, num_layers=2, dropout=0.01):
        super(GRUVAE, self).__init__()
        self.encoder_gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        self.fc_z_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.decoder_gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim=89]
        _, hidden = self.encoder_gru(x) 
        h = hidden[-1]  # [batch_size, hidden_dim]
        mu = self.fc_mu(h)  
        logvar = self.fc_logvar(h)  
        z = self.reparameterize(mu, logvar)  # [batch_size, latent_dim]
        # repeat and feed latent z as input trick
        h0 = self.fc_z_to_hidden(z).unsqueeze(0).repeat(self.encoder_gru.num_layers, 1, 1)  # [num_layers, batch_size, hidden_dim]
        # Initialize decoder input with zeros 
        decoder_input = torch.zeros_like(x)
        output, _ = self.decoder_gru(decoder_input, h0)  # [batch_size, seq_len, hidden_dim]
        x_recon = self.fc_out(output)  # [batch_size, seq_len, input_dim]
        return x_recon, mu, logvar


In [5]:
loaded_scalers = load_scalers("fitted_scalers")
RNN_dataset=ModbusFlowStream( 
    shuffle=False,chunk_size=1,batch_size=64,csv_files=modbus.dataset["benign_dataset_dir"][0:2],
    scalers=loaded_scalers['network-wide']['min_max_scalers'],window_size=30
)
RNN_dataloadder=DataLoader(RNN_dataset,batch_size=1,shuffle=False)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
GRU_VAE_model = GRUVAE().to(device)
lr = 0.01
wd= 1e-4
shuffle_files =True
GRU_VAE_optimizer = optim.Adam(GRU_VAE_model.parameters(), lr=lr, weight_decay=wd)



Successfully loaded scalers for 'network-wide'


In [6]:
for epoch in range(3):
    time_1 = time.time()
    train_loss = 0
    GRU_VAE_model.train()
    if shuffle_files:
        sys_rand = SystemRandom()
        sys_rand.shuffle(RNN_dataset.file_order_indices)
    for sequences, _ in RNN_dataloadder:
        sequences = sequences.squeeze().to(device)
        GRU_VAE_optimizer.zero_grad()
        recon, mu, logvar = GRU_VAE_model(sequences)
        loss = vae_loss_function(recon, sequences, mu, logvar)/sequences.size(0)
        loss.backward()
        GRU_VAE_optimizer.step()
        train_loss += loss.item()
    print("time",time.time()-time_1,f"Epoch {epoch}, Train Loss: {train_loss/len(RNN_dataloadder)}")


time 103.30588173866272 Epoch 0, Train Loss: 127700652.60516566
time 95.60702395439148 Epoch 1, Train Loss: 70232281.69612122
time 94.19640827178955 Epoch 2, Train Loss: 3457994.1813964844
